In [21]:
import pandas as pd
import re

In [22]:
pd.options.display.max_columns = 99

In [23]:
df = pd.read_csv('./data-example.csv')
column_names = list(df)

In [24]:
def word_count(strings):
    words = strings.split(' ')
    return len(words)

In [25]:
def category_count(words):
    counts = dict()
    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1
    return counts

In [26]:
def get_column_info(df, data):
    col_id = None
    col_name = data
    if '.' in data:
        column = data.split(' ')
        col_id = column[0].replace('.','').upper()
        col_name = ' '.join(column[1:-1])
    col_type = str(df[data].dtypes)
    col_category = []
    if col_type == 'object':
        categories = df[data].unique().tolist()
        print(categories)
        for value in categories:
            if type(value) == str:
                if '://' in value or 'UTC' in value or 'www' in value:
                    value = ''
                value = value.replace(' / ','/')
            if type(value) == float:
                pass
            elif '|' in value and type(value) == str:
                categories = value.split('|')
                for cat in categories:
                    col_category.append(cat)
            elif value == '' and value == None:
                pass
            else:
                words = word_count(str(value))
                if words > 30:
                    pass
                else:
                    col_category.append(value)
    if 'link' in data:
        col_category = []
    col_category = '#'.join(col_category)
    col_category = col_category.replace('#',',')
    col_category = col_category.replace('/',',')
    col_category = col_category.replace('),,',')#')
    col_category = col_category.replace(').',')#')
    col_category = col_category.replace('),',')#')
    if '(' in col_category:
        col_category = col_category.split('#')
    else:
        col_category = col_category.split(',')
    category = []
    for cat in col_category:
        if cat == '':
            pass
        else:
            category.append(cat.strip())
    if col_name == 'Unique Response Number':
        category = []
    category = category_count(category)
    col_category = []
    for v in category:
        col_category = col_category
        col_category.append({
            'name':v.strip(','),
            'total': category[v]
        })
    if len(col_category) == 0:
        col_category = None
    return {
        'id': col_id,
        'name': col_name,
        'type': col_type,
        'categories': col_category
    }

In [27]:
col_info = [get_column_info(df, x) for x in column_names]

['545728-545719-53376637', '545728-545719-53649934', '545728-545719-53650563', '545728-545719-53668499', '545728-545719-53741002', '545728-545719-53853866', '545728-545719-53855495', '545728-545719-53915703', '545728-545719-53934963', '545728-545719-53944718', '545728-545719-53947014', '545728-545719-53951949', '545728-545719-53954852', '545728-545719-54002777', '545728-545719-54010262', '545728-545719-54042496', '545728-545719-54043245', '545728-545719-54037064', '545728-545719-54085663', '545728-545719-54099039', '545728-545719-54100737', '545728-545719-54111814', '545728-545719-54146683', '545728-545719-54176682', '545728-545719-54198317', '545728-545719-54120609', '545728-545719-54234908', '545728-545719-54236377', '545728-545719-54236620', '545728-545719-54237211', '545728-545719-54237453', '545728-545719-54237727', '545728-545719-54237935', '545728-545719-54238154', '545728-545719-54238695', '545728-545719-54238987']
['The SeaCleaners', 'The Institute of Environmental Science and

In [12]:
pd.set_option('display.max_row', 100)

In [15]:
cinfo = pd.DataFrame(col_info)

In [20]:
cinfo

,id,name,type,categories
0,None,Unique Response Number,object,None
1,None,Introduction,float64,None
2,1,I agree to share the information provided in t...,float64,None
3,2,I will provide the answers that reflect the be...,float64,None
4,3,I have received adequate information about the...,float64,None
5,4,Your,float64,None
6,4A,Your,float64,None
7,4B,Contact details (preferably e-mail,float64,None
8,4C,Second/alternative contact,float64,None
9,4D,Your organisation (If you are not affiliated w...,object,"[{'name': 'The SeaCleaners,The Institute of En..."


In [19]:
cinfo.iloc[90]['name']

"In the boxes below, please provide i.e. copy and paste links, URL's, website links etc to documents about your action. We are interested in websites, reports, images, media articles etc. Please place only one link per box provided. If you have no documents to share, please write 'I have no links to provide' in the first box. You can also email documents and images - as attachments - to"

In [10]:
pd.DataFrame(col_info).to_json('data-info.json', orient='records')

In [11]:
pd.DataFrame(col_info).to_dict('records')

[{'id': None,
  'name': 'Unique Response Number',
  'type': 'object',
  'categories': None},
 {'id': None, 'name': 'Introduction', 'type': 'float64', 'categories': None},
 {'id': '1',
  'name': 'I agree to share the information provided in this survey. (Note we will not share your name, role or email',
  'type': 'float64',
  'categories': None},
 {'id': '2',
  'name': 'I will provide the answers that reflect the best of my knowledge and facts available to',
  'type': 'float64',
  'categories': None},
 {'id': '3',
  'name': 'I have received adequate information about the survey and about my ethical rights as a participant, including my right to withdraw. I consent to take part in this',
  'type': 'float64',
  'categories': None},
 {'id': '4', 'name': 'Your', 'type': 'float64', 'categories': None},
 {'id': '4A', 'name': 'Your', 'type': 'float64', 'categories': None},
 {'id': '4B',
  'name': 'Contact details (preferably e-mail',
  'type': 'float64',
  'categories': None},
 {'id': '4C',
  